<a href="https://colab.research.google.com/github/whyzhuce/XConparraison/blob/master/Monte_Carlo_dans_le_mod%C3%A8le_SABR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo dans le modèle SABR
----
> **Antonin Chaix**

L'objet de ce TP est d'implémenter un Monte Carlo dans le modèle à volatilité stochastique SABR.

Au moyen de ce Monte Carlo on pourra vérifier :

* que les prix obtenus pour des options européennes sont conformes à ceux fournis par la formule SABR
* que le prix obtenu pour une option binaire européenne est conforme au pricing par call spread (utilisant les vols implicites SABR)


## Le modèle SABR

Le modèle SABR spécifie directement la dynamique du forward de l'actif modélisé (d'où l'absence de drift) :


$$
dF_t = \sigma_t F_t^\beta dW_t^1 \qquad \frac{d\sigma_t}{\sigma_t} = \alpha dW_t^2
$$

$F_0$ étant donné, le modèle est défini par les 4 paramètres :
* $\sigma_0$ (valeur initiale de la volatilité)
* $\alpha$ (volatilité de la volatilité)
* $\beta$ (exposant CEV)
* $\rho$ (correlation entre le forward et sa volatilité, i.e. entre $W^1$ et $W^2$)

et on dispose d'une formule approchée (implémentée dans la suite) nous donnant dans le modèle SABR la valeur de la vol implicite d'un call/put de maturité $T$ et de strike $K$ en fonction des paramètres $F_0$, $σ_0$, $α$, $β$ et $ρ$.


## Monte Carlo sous SABR

C'est tout simple : étant donnés $F_t$ et $\sigma_t$ à l'instant $t$, leurs valeurs au temps suivant $t+\Delta t$ sont simulées de la façon suivante, au moyen de deux variables gaussiennes $\varepsilon_1$ et $\varepsilon_2$ centrées réduites :

$$
F_{t+\Delta t} = F_t + \sigma_t F_t^\beta \sqrt{\Delta t} \,\varepsilon_1\,\qquad\text{et}\qquad
\sigma_{t+\Delta t} = \sigma_t\exp\left(-\frac{\alpha^2}{2}\Delta t + α \sqrt{\Delta t}\left(\rho \varepsilon_1 + \sqrt{1-\rho^2}\,\varepsilon_2\right)\right)
$$

Dans ce qui suit on s'intéresse à un actif $S$ ne distribuant pas de dividendes. On fera par ailleurs l'hypothèse d'un taux $r$ constant.

**Le bloc de code suivant permet de simuler les trajectoires de $F$ et $\sigma$ :**

In [ ]:
import numpy as np
import math
from scipy.stats import norm

# options maturity = MC horizon
T = 1

# underlying params
S0 = 100
r = 0.015

# MC params
Nsimul = 10**6
Nsteps = 100

# SABR params
sigma0 = 1.9847
alpha = 0.2657
rho = -0.3660
beta =  0.5000

# precalcs
dt = T / Nsteps
sqrt_dt = math.sqrt(dt)
discount = math.exp(-r * T)

# generate gaussian random variables (with correl)
norm1 = np.random.normal(0, 1, (Nsteps, Nsimul))
norm2 = rho * norm1 + math.sqrt(1 - rho ** 2) * np.random.normal(0, 1, (Nsteps, Nsimul))

# forward on time steps x simulations
fwd = np.empty(shape=(Nsteps+1, Nsimul))
fwd[0,:] = S0 / discount

# vol on time steps x simulations
sigma = np.empty(shape=(Nsteps+1, Nsimul))
sigma[0,:] = sigma0

# SABR MC loop
for j in range(Nsteps) :
    fwd[j+1,:] = fwd[j,:] + sigma[j,:] * np.power(fwd[j,:], beta) * sqrt_dt * norm1[j,:]
    sigma[j+1,:] = sigma[j,:] * np.exp(-0.5 * alpha**2 * dt + alpha * sqrt_dt * norm2[j,:])


print("INPUTS | {:,} simulations | {} time steps | T = {} | \n\n".format(Nsimul, Nsteps, T).replace(',', ' '))

INPUTS | 1 000 000 simulations | 100 time steps | T = 1 | 




## Fonctions utiles

Ci-dessous une petite fonction permettant de calculer/afficher prix et intervalle de confiance dans le Monte Carlo SABR en fonction du payoff de l'option, la formule de Black pour un call et enfin la formule SABR donnant la vol implicite d'un call/put de maturité $T$ et de strike $K$ en fonction des paramètres $F_0$, $σ_0$, $α$, $β$ et $ρ$.

In [ ]:
# compute and display price & confidence interval
def compute_and_print_price(payoff, option_name, closed_form_price = -1) :
	price = payoff.mean()
	stdev = np.std(payoff)
	IClow = price - 1.96 * stdev / math.sqrt(Nsimul)
	ICup = price + 1.96 * stdev / math.sqrt(Nsimul)
	if (closed_form_price == -1) : print("{} = {:.4f}".format(option_name, price) )
	else : 	print("{} = {:.4f} (closed-form price = {:.4f})".format(option_name, price, closed_form_price) )
	print("IC 95% = [{:.4f} ; {:.4f}]".format(IClow,ICup) )
	print("-----------------------------------------------------")

# BS formula for a call
def bs_call(T, K, F0, sigma) :
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(F0/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return F0 * norm.cdf(d1) - K * norm.cdf(d2)


# SABR lognormal vol formula from [https://github.com/ynouri/pysabr] made numpy compatible
def sabr_vol (T, K, F0, sigma0, alpha, rho, beta) :
    """
    Hagan's 2002 SABR lognormal vol expansion.
    The strike K can be a scalar or an array, the function will return an array
    of lognormal vols.
    """
    eps = 1e-07
    logfk = np.log(F0 / K)
    fkbeta = (F0*K)**(1 - beta)
    a = (1 - beta)**2 * sigma0**2 / (24 * fkbeta)
    b = 0.25 * rho * beta * alpha * sigma0 / fkbeta**0.5
    c = (2 - 3*rho**2) * alpha**2 / 24
    d = fkbeta**0.5
    v = (1 - beta)**2 * logfk**2 / 24
    w = (1 - beta)**4 * logfk**4 / 1920
    z = alpha * fkbeta**0.5 * logfk / sigma0
    tmp = sigma0 * (1 + (a + b + c) * T)
    num = np.where(abs(z) > eps, z * tmp, tmp)
    den = np.where(abs(z) > eps, (d * (1 + v + w) * _x(rho, z)), d * (1 + v + w))
    return num / den

def _x(rho, z):
    """Return function x used in Hagan's 2002 SABR lognormal vol expansion."""
    a = (1 - 2*rho*z + z**2)**.5 + z - rho
    b = 1 - rho
    return np.log(a / b)

## Evaluation Monte Carlo vs. formules analytiques

Il ne reste plus qu'à définir le strike de nos options et à calculer leur payoff.

In [ ]:
# call & digital strike
K = 100

# compute payoffs for Monte Carlo
payoff_call = discount * np.maximum(fwd[-1,:] - K , 0)  # => MC call price  = payoff_call.mean()
payoff_binaire = discount * np.where(fwd[-1,:] > K, 1., 0.)  # => MC binaire price  = payoff_binaire.mean()

# closed form for european call
F0 = S0 / discount
vol = sabr_vol (T, K, F0, sigma0, alpha, rho, beta)
call_price =  discount * bs_call(T, K, F0, vol)

# closed form for digital option (call spread)
vol_minus = sabr_vol (T, K-1, F0, sigma0, alpha, rho, beta)
vol_plus  = sabr_vol (T, K+1, F0, sigma0, alpha, rho, beta)
digital = discount * (bs_call(T, K-1, F0, vol_minus) - bs_call(T, K+1, F0, vol_plus)) / 2.
digital_bs = discount * (bs_call(T, K-1, F0, vol) - bs_call(T, K+1, F0, vol)) / 2. # binaire naïve sans prendre en compte le skew

# compute & display MC prices
compute_and_print_price(fwd[-1,:], "Forward", F0)
compute_and_print_price(payoff_call, "Call européen", call_price)
compute_and_print_price(payoff_binaire, "Binaire", digital)
print("Binaire BS naïve = {:.4f}".format(digital_bs))
print("-----------------------------------------------------")

print("\nIt works !")



Forward = 101.5060 (closed-form price = 101.5113)
IC 95% = [101.4665 ; 101.5455]
-----------------------------------------------------
Call européen = 8.6316 (closed-form price = 8.6337)
IC 95% = [8.6077 ; 8.6554]
-----------------------------------------------------
Binaire = 0.5220 (closed-form price = 0.5222)
IC 95% = [0.5210 ; 0.5229]
-----------------------------------------------------
Binaire BS naïve = 0.4831
-----------------------------------------------------

It works !
